In [6]:
# Required imports
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QFileDialog, QVBoxLayout, QGraphicsView, QGraphicsScene, QTextBrowser, QScrollArea, QLineEdit, QLabel, QHBoxLayout, QGridLayout, QCheckBox
from PyQt5.QtGui import QPen, QColor
from PyQt5.QtCore import Qt
import ezdxf
import os
import qdarktheme

# Parses input file to extract entities relative to gcode
def parse_dxf_file(file_path):
    parsed_data = []
    doc = ezdxf.readfile(file_path)
    for entity in doc.modelspace():
        if entity.dxftype() == 'LINE':
            start_point = entity.dxf.start
            end_point = entity.dxf.end
            parsed_data.append({
                'type': 'LINE',
                'start_point': (start_point.x, start_point.y),
                'end_point': (end_point.x, end_point.y)
            })
        elif entity.dxftype() == 'CIRCLE':
            center = entity.dxf.center
            radius = entity.dxf.radius
            parsed_data.append({
                'type': 'CIRCLE',
                'center_point': (center.x, center.y),
                'radius': radius
            })
        elif entity.dxftype() == 'ARC':
            center = entity.dxf.center
            radius = entity.dxf.radius
            start_angle = entity.dxf.start_angle
            end_angle = entity.dxf.end_angle
            parsed_data.append({
                'type': 'ARC',
                'center_point': (center.x, center.y),
                'radius': radius,
                'start_angle': start_angle,
                'end_angle': end_angle
            })
        elif entity.dxftype() == 'LWPOLYLINE':
            polyline_data = []
            for vertex in entity.points():
                polyline_data.append((vertex.x, vertex.y))
            parsed_data.append({
                'type': 'POLYLINE',
                'vertices': polyline_data
            })

    return parsed_data

# block num space padding logic
def blockNumPad(blockNum):
    blockNumStr = ""
    if blockNum < 10:
        blockNumStr = "    0" + str(blockNum)
    elif blockNum < 100:
        blockNumStr = "    " + str(blockNum)
    else:
        blockNumStr = "   " + str(blockNum)
    return blockNumStr

# Parses DXF data into Emco supported GCode
def generate_gcode_from_dxf(parsed_data, isUseM3M5Checked):
    # starting blocks
    gcode = []
    blockNum = 0
    gcode.append(f'%\n')
    gcode.append(f'    N` G`   X `    Z `  F`  H\n')
    if isUseM3M5Checked:
        gcode.append(f'{blockNumPad(blockNum)}M03\n')

    # generate main gcode blocks
    blockNum +=1
    for entity in parsed_data:
        toAppend = ""
        if entity['type'] == 'LINE':
            start_x, start_y = entity['start_point']
            end_x, end_y = entity['end_point']
            gcodeToAdd = (f'01 X{end_x:.3f} Y{end_y:.3f}')
        elif entity['type'] == 'CIRCLE':
            center_x, center_y = entity['center_point']
            radius = entity['radius']
            gcodeToAdd = (f'02 X{center_x:.3f} Y{center_y:.3f} R{radius:.3f}')
            gcodeToAdd = (f'03 X{center_x:.3f} Y{center_y:.3f} R{radius:.3f}')
        elif entity['type'] == 'ARC':
            center_x, center_y = entity['center_point']
            radius = entity['radius']
            start_angle = entity['start_angle']
            end_angle = entity['end_angle']
            if start_angle < end_angle:
                gcodeToAdd = (f'M02 X{center_x:.3f} Y{center_y:.3f} R{radius:.3f} A{start_angle:.3f} B{end_angle:.3f}')
            else:
                gcodeToAdd = (f'M03 X{center_x:.3f} Y{center_y:.3f} R{radius:.3f} A{start_angle:.3f} B{end_angle:.3f}')
        elif entity['type'] == 'POLYLINE':
            vertices = entity['vertices']
            for vertex in vertices:
                x, y = vertex
                gcodeToAdd = (f'01 X{x:.3f} Y{y:.3f}')
            
        
        # append to output
        toAppend = blockNumPad(blockNum) + " " + gcodeToAdd + "\n"
        gcode.append(toAppend)
        blockNum += 1
           
    # ending blocks
    if isUseM3M5Checked:
        gcode.append(f'{blockNumPad(blockNum)}M05\n')
        blockNum +=1
    gcode.append(f'{blockNumPad(blockNum)}M30\n')
    gcode.append(f'   M\n')
    return gcode

# Determins the max DXF size for scaling DXF preview 
def calculate_drawing_extents(entities):
    min_x = float('inf')
    min_y = float('inf')
    max_x = float('-inf')
    max_y = float('-inf')

    for entity in entities:
        if entity['type'] == 'LINE':
            start_x, start_y = entity['start_point']
            end_x, end_y = entity['end_point']
            min_x = min(min_x, start_x, end_x)
            max_x = max(max_x, start_x, end_x)
            min_y = min(min_y, start_y, end_y)
            max_y = max(max_y, start_y, end_y)
        elif entity['type'] == 'CIRCLE':
            center_x, center_y = entity['center_point']
            radius = entity['radius']
            min_x = min(min_x, center_x - radius)
            max_x = max(max_x, center_x + radius)
            min_y = min(min_y, center_y - radius)
            max_y = max(max_y, center_y + radius)
        elif entity['type'] == 'ARC':
            center_x, center_y = entity['center_point']
            radius = entity['radius']
            start_angle = entity['start_angle']
            end_angle = entity['end_angle']
            min_x = min(min_x, center_x - radius)
            max_x = max(max_x, center_x + radius)
            min_y = min(min_y, center_y - radius)
            max_y = max(max_y, center_y + radius)
        elif entity['type'] == 'POLYLINE':
            for vertex in entity['vertices']:
                x, y = vertex
                min_x = min(min_x, x)
                max_x = max(max_x, x)
                min_y = min(min_y, y)
                max_y = max(max_y, y)

    return min_x, min_y, max_x, max_y

# Emco Processor GUI
class DXFParserGUI(QWidget):
    
    def __init__(self):
        super().__init__()
        self.initUI()
        self.file_path = ""
        self.output_code = ""

    def initUI(self):
        layout = QVBoxLayout()

        # Display the DXF drawing
        self.view = QGraphicsView(self)
        self.scene = QGraphicsScene()
        self.view.setScene(self.scene)
        layout.addWidget(self.view)

        # Open DXF file button
        self.btn_open = QPushButton("Open DXF File")
        self.btn_open.clicked.connect(self.openFile)
        layout.addWidget(self.btn_open)

        # Create a grid layout for labels and text boxes
        grid_layout = QGridLayout()

        # Stock Radius input
        self.stock_radius_label = QLabel("Stock Radius (mm):")
        self.stock_radius_input = QLineEdit()
        grid_layout.addWidget(self.stock_radius_label, 0, 0)
        grid_layout.addWidget(self.stock_radius_input, 0, 1)

        # Roughing Feedrate input
        self.roughing_feedrate_label = QLabel("Roughing Feedrate (mm/min):")
        self.roughing_feedrate_input = QLineEdit()
        grid_layout.addWidget(self.roughing_feedrate_label, 0, 3)
        grid_layout.addWidget(self.roughing_feedrate_input, 0, 4)

        # Roughing Stepdown input
        self.roughing_stepdown_label = QLabel("Roughing Stepdown (mm):")
        self.roughing_stepdown_input = QLineEdit()
        grid_layout.addWidget(self.roughing_stepdown_label, 0, 5)
        grid_layout.addWidget(self.roughing_stepdown_input, 0, 6)

        # Finishing Feedrate input
        self.finishing_feedrate_label = QLabel("Finishing Feedrate (mm/min):")
        self.finishing_feedrate_input = QLineEdit()
        grid_layout.addWidget(self.finishing_feedrate_label, 0, 7)
        grid_layout.addWidget(self.finishing_feedrate_input, 0, 8)

        # Finishing Stepdown input
        self.finishing_stepdown_label = QLabel("Finishing Stepdown (mm):")
        self.finishing_stepdown_input = QLineEdit()
        grid_layout.addWidget(self.finishing_stepdown_label, 0, 9)
        grid_layout.addWidget(self.finishing_stepdown_input, 0, 10)
        
        # Use M3/M5 toggle button
        self.use_m3_m5_checkbox = QCheckBox("Use M3/M5")
        grid_layout.addWidget(self.use_m3_m5_checkbox, 1, 0, 1, 1)

        # Add the grid layout to the main layout
        layout.addLayout(grid_layout)

        # Scrollable G-code display
        self.gcode_browser = QTextBrowser(self)
        self.scroll_area = QScrollArea(self)
        self.scroll_area.setWidget(self.gcode_browser)
        self.gcode_browser.setFixedWidth(1735)
        self.gcode_browser.setFixedHeight(575)
        layout.addWidget(self.scroll_area)
        
        # Set the font size for the QTextBrowser
        font = self.gcode_browser.currentFont()
        font.setPointSize(14)  # Adjust the font size (in points) as needed
        self.gcode_browser.setFont(font)
        
        # Create a grid layout for generate and save buttons
        grid_layout2 = QGridLayout()

        # Generate G-code button
        self.btn_gen = QPushButton("Generate G-code")
        self.btn_gen.clicked.connect(self.generateGCode)
        grid_layout2.addWidget(self.btn_gen, 0, 0)
        
        # Save G-code button
        self.btn_save = QPushButton("Save G-code")
        self.btn_save.clicked.connect(self.saveGCode)
        grid_layout2.addWidget(self.btn_save, 0, 1)
        
        # Add the grid layout to the main layout
        layout.addLayout(grid_layout2)

        self.setLayout(layout)
        self.setGeometry(100, 100, 1600, 1200)
        self.setWindowTitle("DXF Parser")
        self.show()
        
##############################################################################################
#################################  CALLBACK FUNCTIONS  #######################################
##############################################################################################

    # Function to get the value of the Use M3/M5 checkbox
    def isUseM3M5Checked(self):
        return self.use_m3_m5_checkbox.isChecked()
    
    # Opens DXF file from your computer
    def openFile(self):
        options = QFileDialog.Options()
        self.file_path, _ = QFileDialog.getOpenFileName(self, "Open DXF File", "", "DXF Files (*.dxf);;All Files (*)", options=options)
        if self.file_path:
            entities = parse_dxf_file(self.file_path)
            self.parseAndDisplayDXF(entities)
           
    # Starts gcode generating process
    def generateGCode(self):
        entities = parse_dxf_file(self.file_path)
        self.output_code = generate_gcode_from_dxf(entities, self.isUseM3M5Checked())
        self.gcode_browser.clear()
        self.gcode_browser.append(''.join(self.output_code))
            
    # Displays DXF and scales uniformly to fit screen
    def parseAndDisplayDXF(self, entities):
        self.scene.clear()
        # Get the drawing extents for scaling
        min_x, min_y, max_x, max_y = calculate_drawing_extents(entities)
        drawing_width = max_x - min_x
        drawing_height = max_y - min_y
        scale_factor = 1000 / max(drawing_width, drawing_height)  # Adjust the scale as needed

        for entity in entities:
            if entity['type'] == 'LINE':
                start_x, start_y = entity['start_point']
                end_x, end_y = entity['end_point']
                line = self.scene.addLine(start_x * scale_factor, -start_y * scale_factor, end_x * scale_factor, -end_y * scale_factor)
                pen = QPen(QColor(0, 0, 255))
                line.setPen(pen)
            elif entity['type'] == 'CIRCLE':
                center_x, center_y = entity['center_point']
                radius = entity['radius']
                circle = self.scene.addEllipse((center_x - radius) * scale_factor, (-center_y - radius) * scale_factor, radius * 2 * scale_factor, radius * 2 * scale_factor)
                pen = QPen(QColor(0, 0, 255))
                circle.setPen(pen)
            elif entity['type'] == 'POLYLINE':
                vertices = entity['vertices']
                polyline = QGraphicsView()
                poly_scene = QGraphicsScene()
                polyline.setScene(poly_scene)
                pen = QPen(QColor(0, 0, 255))
                for i in range(len(vertices) - 1):
                    start_x, start_y = vertices[i]
                    end_x, end_y = vertices[i + 1]
                    line = poly_scene.addLine(start_x * scale_factor, -start_y * scale_factor, end_x * scale_factor, -end_y * scale_factor)
                    line.setPen(pen)
                self.view.setScene(self.scene)

    # Saves gcode file to your computer
    def saveGCode(self):
        filename, _ = QFileDialog.getSaveFileName(self, filter="*.cnc")
        if filename:
            f = open(filename, 'w')
            toWrite = ""
            toWrite = toWrite.join(self.output_code) 
            f.write(toWrite)
            self.setWindowTitle(str(os.path.basename(filename)) + " - Notepad Alpha")
            f.close()
       
def main():
    app = QApplication(sys.argv)
    qdarktheme.setup_theme()
    window = DXFParserGUI()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

SystemExit: 0